In [ ]:
import json

In [ ]:
!pwd

In [ ]:
with open('../data/articles.json') as json_file:
    data = json.load(json_file)

In [ ]:
number = 0
for i in data:
    number += len(data[i]["places_and_poi"])
print(number)

In [ ]:
import time
import grequests

In [ ]:
# 20'000 credits daily limit per application (identified by the parameter 'username'),
# the hourly limit is 1000 credits. A credit is a web service request hit for most services.
# An exception is thrown when the limit is exceeded. 
geonames = {"exact_match":"http://secure.geonames.org/searchJSON?maxRows=30&username=kwh44&name_equals=",
            "general_match": "http://secure.geonames.org/searchJSON?maxRows=30&username=kwh44&name="}

In [ ]:
# https://www.gisgraphy.com/documentation/user-guide.php#fulltextservice
gisgraphy = "https://services.gisgraphy.com/fulltext/fulltextsearch?&format=json&allwordsrequired=true&q="

In [ ]:
# https://github.com/dbpedia/dbpedia-lookup
dbpedia = "https://lookup.dbpedia.org/api/search?&typeName=place&format=JSON_FULL&maxResults=3&query=Cain%27s%20Ballroom"
resource_name = "Cain's_Ballroom"
rdf_page_json = f"https://dbpedia.org/data/{resource_name}.json"

In [ ]:
# making multiple queries to gazetteer api
urls = []
s = time.time()
rs = (grequests.get(u) for u in urls)
result = grequests.map(rs)
e = time.time()
print("Lookup time: ", e - s)

##### query placename string and response placename string : similarity measure

In [ ]:
import difflib
def similar(a: str, b: str) -> float:
    return round(difflib.SequenceMatcher(lambda x: x == " ", a, b).ratio(), 3)

In [ ]:
a = "Hudson River Historic District"
b = "Hudson River National Historic Landmark District"
print(similar(a, b))

In [ ]:
# article webpage preprocessing

In [ ]:
import html2text
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.nationalgeographic.com/travel/article/geek-retreat-the-best-of-silicon-valley"

In [ ]:
def text_from_html(url): #data_folder_html_file_id):
    
    #road_trip = open("../data/" + str(data_folder_html_file_id) + ".html", "r").read()
    road_trip = requests.get(url).text
    soup = BeautifulSoup(road_trip)
    links = []
    for link in soup.findAll('a'):
        http_link = link.get('href')
        if "nationalgeographic" in http_link:
            continue
        if not http_link.startswith("http"):
            continue
        if len(http_link) > 5:
            links.append(http_link)
    
    text_maker = html2text.HTML2Text()
    text_maker.ignore_links = True
    text_maker.ignore_images = True
    road_trip_processed = text_maker.handle(road_trip)
    
    start = "ShareTweetEmail"
    index = road_trip_processed.find(start)
    road_trip_processed = road_trip_processed[index + len(start) + 1:]
    
    end = road_trip_processed.find("ShareTweetEmail")
    road_trip_processed = road_trip_processed[:end]
    road_trip_processed = road_trip_processed.replace("\n\n", ". ")
    for i in ["\n", "[", "]", "*", "#", "_", "\\'", '\\']:
        road_trip_processed = road_trip_processed.replace(i, " ")
    road_trip_processed = road_trip_processed.replace('>', ", ")
    for i in range(4): # 4 seems enough
        road_trip_processed = road_trip_processed.replace("  ", " ")
    road_trip_processed = road_trip_processed.replace("..", ".")
               
    return road_trip_processed.strip(), links 

In [ ]:
text, links = text_from_html(url)

In [ ]:
links

In [ ]:
text

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# UNDERSTANDING ENTITY TYPES
{
    "FAC": spacy.explain("FAC"),
    "GPE": spacy.explain("GPE"),
    "LOC": spacy.explain("LOC"),
    "ORG": spacy.explain("ORG")
}

In [ ]:
nlp = spacy.load('en_core_web_trf')

In [ ]:
%time displacy.render(nlp(text), style='ent', jupyter=True)

In [ ]:
def get_ne(txt):
    ne_types = ["GPE", "FAC", "LOC", "ORG"]
    ne = nlp(text).ents
    target_ne = {str(ne[i]).strip(): i for i in range(len(ne)) if ne[i].label_ in ne_types}
    return target_ne

In [ ]:
%time ne = get_ne(text)

In [ ]:
len(ne)

In [ ]:
# existence detection

In [ ]:
entities_extracted = list(ne.keys())

In [ ]:
len(entities_extracted)

In [ ]:
extracted_ents_urls = [gisgraphy["api"] + i + gisgraphy["format"] for i in entities_extracted]

In [ ]:
extracted_ents_urls

In [ ]:
s = time.time()
rs = [grequests.get(u) for u in extracted_ents_urls]
result = (grequests.map(rs))
e = time.time()

In [ ]:
print("Lookup time: ", e - s)